# Librerias

In [13]:
import numpy as np
import pandas as pd

import boto3
import cv2
import re
import difflib
import enchant

import datetime
import time
import os

from unicodedata import normalize
from extract_key_value import extract_column_year
from text_detection import get_aws_analyze_document

# Funciones

In [14]:
def df_text_to_string(df_text):
    df_text = df_text.applymap(str)
    string= ''
    for i, row in df_text.iterrows():
        line_text = ''.join(row)
        string += line_text +'\n'
    return string

def list_dfs_to_string(list_df_text):
    String = ' '.join([df_text_to_string(x) for x in list_df_text])
    return String

def clean_text(s):
    s = str(s)
    s = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", s), 0, re.I
    )
    s = normalize('NFC', s)
    minus = s.lower()
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in minus:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

def extract_dates(s):
    try:
        mesesDic = {
            'enero': '01',
            'febrero': '02',
            'marzo': '03',
            'abril': '04',
            'mayo': '05',
            'junio': '06',
            'julio': '07',
            'agosto': '08',
            'septiembre': '09',
            'octubre': '10',
            'noviembre': '11',
            'diciembre': '12'
        }
        dates = re.finditer('(?P<day>[0-9]{2}) (de) (?P<month>[a-zA-Z]+) (de|del) (?P<year>[0-9]{4})',s)
        list_dates = []
        for x in dates:
            day = x.group('day')
            month = mesesDic[x.group('month')]
            year = x.group('year')
            fecha = datetime.date(int(year),int(month),int(day))
            list_dates.append(fecha)
        list_dates.sort(reverse=True)
        list_dates = [y.strftime('%d-%m-%Y') for y in list_dates]
        return list_dates[0]
    except:
        return None

def extract_units(s):
    try:
        unidades = re.finditer('((miles|millones) (de) (?P<moneda>(dolares|pesos|usd)( colombianos)?))',s,flags=2)
        i = 0
        for x in unidades:
            if i ==0:
                moneda1 = x.group()
            moneda = x.group('moneda')
            if moneda == 'usd':
                return x.group()
            if moneda == 'dolares':
                return x.group()
        return moneda1
    except:
        return None
    
def dates_in_tables_f(list_df_tables):
    dates_in_tables =[]
    for df_tables in list_df_tables:
        s = df_text_to_string(df_tables)
        date = extract_dates(s)    
        dates_in_tables.append(date)
    dates_in_tables = [x for x in dates_in_tables if x]
    dates_in_tables.sort()
    date = None
    if len(dates_in_tables) > 0:
        date = dates_in_tables[-1]
    return date

def intenta_limpiar_texto(x):
    try:
        y=clean_text(x)
        return y
    except:
        return x

def loc_variable_key(list_df_tables,variable):
    lis_dict = []
    for hoja in range(len(list_df_tables)):
        for col in range(list_df_tables[hoja].shape[1]):
            for i in range(list_df_tables[hoja].shape[0]):
                    try:
                        s =list_df_tables[hoja].iloc[i,col]
                        var = re.search('|'.join(v_contables_dict[variable]),s)
                        if var:
                            d_ubi = {s:[hoja, col, i]}
                            lis_dict.append(d_ubi)
                    except:
                        pass
    return lis_dict

# Integración

In [37]:
v_contables_dict = {
    'Caja y Bancos':[
        'caja y bancos', 
        'efectivo y equivalentes en efectivo',
        'efectivo y equivalentes de efectivo',
    ],
    'Total activo':[
        'total activo',
        'suma de los activos', 
        'activo total'
    ],
    'Total pasivo':[
        'total pasivo',
        'suma de los pasivos',
        'pasivo total',
        'total pasivos'

    ],
    'Total patrimonio':[
        'total patrimonio',
        'suma de los patrimonios',
        'patrimonio total',
        'patrimonio',
        'suma del capital contable',
        'total capital contable', 
        'total capital', 
    ],
    'Ventas':[
        'ventas',  
        'ventas por operacion ordinacia',
        'ingresos por operacion',
        'ingresos por operacion ordinaria', 
        'ingresos operacionales', 
        'ventas brutas', 
        'ingreso por actividades ordinarias',
        'total ingresos operacionales',
        'ingresos por ventas'
    ],
    'Costo de ventas':[
        'costos de ventas', 
        'costos por ventas', 
        'costo de actividades ordinarias',
    ],
    'Utilidad Bruta':[
        'utilidad bruta',
        'perdida bruta',
    ],
    'utilidad operacional': [
        'utilidad operacional', 
        'perdida operacional',
    ],
    'utilidad antes de impuestos': [
        'utilidad antes de impuestos', 
        'perdida antes de impuestos',
    ],
    'utilidad neta': [
        'utilidad neta', 
        'perdida neta',
        'utilidad neta del periodo'
    ],
}

In [38]:
def get_output_indexes(list_words):
    output_indexes = {}
    for v_contables_key in v_contables_dict.keys():
        row = -1
        for similar_word in v_contables_dict[v_contables_key]:
            close_matches = difflib.get_close_matches(similar_word, list_words, n=3)
            if len(close_matches) > 0:
                row = list_words.index(close_matches[0])
                break
        output_indexes[v_contables_key] = row
    return output_indexes

def Calacas_chidas_AI(bucket, bucket_file):
    file_path = os.path.join('DownTest', bucket_file)
    if bucket_file not in os.listdir('DownTest'):
        boto3.client('s3').download_file(
            bucket, 
            file_path, 
            file_path
        )
    list_df_tables, list_df_text = get_aws_analyze_document(file_path)

    Text = clean_text(list_dfs_to_string(list_df_text))
    Unidades = extract_units(Text)
    Fecha = extract_dates(Text)
    if not Fecha:
        Fecha = dates_in_tables_f(list_df_tables)
    
    """
    for x in range(len(list_df_tables)):
        list_df_tables[x] = list_df_tables[x].applymap(clean_text)
    """    
    l=[]
    for x in range(len(list_df_tables)):
        clave = extract_column_year(list_df_tables[x])
        l.append(list_df_tables[x].rename(columns={1:'Clave',clave[1]:'Valor'})[['Clave','Valor']])
    
    df = pd.concat(l,axis=0, ignore_index=True)
    df.replace({'':np.nan},inplace=True)
    df.dropna(inplace=True)
    
    ids_campos = get_output_indexes(df['Clave'].to_list())
    claves = ids_campos.keys()
    
    aux = []
    aux.append(['Fecha',Fecha])
    aux.append(['Unidades en la que se mide',Unidades])
    for i in claves:
        if ids_campos[i] == -1:
            aux.append([i,np.nan])
        else:
            aux.append([i,df.iloc[ids_campos[i]]['Valor']])
    df = pd.DataFrame(aux)    
    return(df)

In [39]:
def get_output_indexes(list_words):
    output_indexes = {}
    for v_contables_key in v_contables_dict.keys():
        row = -1
        all_close_matches = {}
        for similar_word in v_contables_dict[v_contables_key]:
            close_matches = difflib.get_close_matches(similar_word, list_words, n=3)
            for match in close_matches:
                distance = enchant.utils.levenshtein(similar_word, match)
                all_close_matches[match] = distance

            
        if len(all_close_matches) > 0:
            # Sort distances
            all_close_matches = sorted(all_close_matches.items(), key=lambda x: x[1])
            # Select best option
            row = list_words.index(all_close_matches[0][0])
            # Duplicated indexes

        output_indexes[v_contables_key] = row
    return output_indexes

In [40]:
bucket = 'calacaschidas'
bucket_file = 'Doc1.pdf'
output = Calacas_chidas_AI(bucket, bucket_file)
final_df = output.set_index(0).T
final_df

,Fecha,Unidades en la que se mide,Caja y Bancos,Total activo,Total pasivo,Total patrimonio,Ventas,Costo de ventas,Utilidad Bruta,utilidad operacional,utilidad antes de impuestos,utilidad neta
1,None,millones de pesos colombianos,"7,075,758","135,448,018","75,103,896","60,344,122","70,846,769","(44,957,508)","25,889,261","20,415,132","14,502,499","14,502,499"
